# Model group generators

Model groups can be configured with object placement engines and spatial constraints to generate a group of models to be added to a `ModelGroup` set. 

`ModelGroup` entities can keep a group of models under an unique frame and easily spawn the models with respect to the model group frame. 
The model group can then be spawned in Gazebo as **multiple individual models** or **nested models**.

In this example, a crate is created from a mesh and filled with **spheres**, **cuboids** and **cylinders** of random dimensions and masses.

In [1]:
import os
from pcg_gazebo.generators import ModelGroupGenerator

generator = ModelGroupGenerator('full_crate')

line 93.
line 104.
line 105.
line 109.
line 111.
line 109.
line 111.
line 106.
line 107.
line 109.
line 111.
line 109.
line 111.
line 108.
line 94.
line 101.
line 102.
line 104.
line 107.
line 109.
line 111.
line 109.
line 111.
line 108.
line 103.
line 95.
line 96.


Add the single-link model of a crate generated from the mesh below to the assets manager.

![crate](images/crate.png)

In [2]:
generator.add_asset(
    tag='crate',
    description=dict(
        type='mesh',
        args=dict(
            visual_mesh='file://' + os.path.abspath('meshes/crate.stl'),
            visual_mesh_scale=[1, 1, 1],
            use_approximated_collision=False,
            name='crate',
            color='xkcd'
        )
    )
)

line 111.


True

Add **spheres**, **cuboids** and **cylinder** models to the assets manager to fill the crate.

Since the dimensions and masses are defined by **lambda functions**, the respective **model factory** for the model primitives is going to be called and the lambda functions run to set the parameters of the model.

In [3]:
generator.add_asset(
    tag='crate_ball',
    description=dict(
        type='sphere',
        args=dict(
            radius="max(0.05, 0.1 * __import__('numpy').random.random())",
            name='sphere',
            mass="max(0.1, __import__('numpy').random.random())",
            color='xkcd'
        )
    )
)

generator.add_asset(
    tag='crate_cuboid',
    description=dict(
        type='box',
        args=dict(
            size="0.1 * __import__('numpy').random.random(3)",
            name='cuboid',
            mass="max(0.01, __import__('numpy').random.random())",
            color='xkcd'
        )
    )
)

generator.add_asset(
    tag='crate_cylinder',
    description=dict(
        type='cylinder',
        args=dict(
            length="max(0.05, 0.1 *__import__('numpy').random.random())",
            radius="max(0.05, 0.1 *__import__('numpy').random.random())",
            name='cuboid',
            mass="max(0.01, __import__('numpy').random.random())",
            color='xkcd'
        )
    )
)

crate_assets = ['crate_ball', 'crate_cuboid', 'crate_cylinder']

line 111.
line 111.
line 111.


Add constraint to place objects tangent to the ground plane.

In [4]:
generator.add_constraint(
    name='tangent_to_ground_plane',
    type='tangent',
    frame='world',
    reference=dict(
        type='plane',
        args=dict(
            origin=[0, 0, 0],
            normal=[0, 0, 1]
        )
    )
)

True

Add a workspace above the crate where the filling objects should be spawned on top of the crate.

In [5]:
generator.add_constraint(
    name='crate_base',
    type='workspace',
    geometry_type='area',
    frame='world',
    points=[ 
              [-0.5, -0.4, 0],
              [-0.5, 0.4, 0],
              [0.5, 0.4, 0],
              [0.5, -0.4, 0]
          ]
)

True

Add a fixed-pose engine to place the crate on the ground plane.

In [6]:
generator.add_engine(
    tag='crate_engine',
    engine_name='fixed_pose',
    models=['crate'],
    poses=[
        [0, 0, 0, 0, 0, 0]
    ],
    constraints=[
        dict(
            model='crate',
            constraint='tangent_to_ground_plane'
        )
    ]
)

True

Add engine to randomily spawn the spheres, cuboids and cylinder above the crate. 

A collision checker makes sure none of the objects overlap or are within another.

In [7]:
num_instances = dict()
for tag in crate_assets:
    num_instances[tag] = 4

generator.add_engine(
    tag='fill_crate',
    engine_name='random_pose',
    models=crate_assets,
    max_num=num_instances,
    model_picker='random',
    no_collision=True,
    policies=[
        dict(
            models=crate_assets,
            config=[
                dict(
                    dofs=['x', 'y'],
                    tag='workspace',
                    workspace='crate_base'
                ),
                dict(
                    dofs=['z'],
                    tag='uniform',
                    mean=0.5,
                    min=0.0,
                    max=3.0
                ),
                dict(
                    dofs=['roll', 'pitch', 'yaw'],
                    tag='uniform',
                    mean=0,
                    min=-3.141592653589793,
                    max=3.141592653589793
                )
            ]
        )
    ]
)

True

Every time the model generator is run, all the assets that have dynamic parameters are re-generated. This makes it possible to have a single configuration for multiple model groups.

In [8]:
# Generate model groups
models = list()
for i in range(3):
    models.append(generator.run(group_name='full_crate_{}'.format(i)))
    print('Full crate model #{} generated, # nested models={}'.format(i, models[-1].n_models))

line 111.
line 1.
line 2.
line 3.
line 4.
line 5.
line 6.
line 7.
crate_engine
line 8.
<class 'pcg_gazebo.generators.engines.fixed_pose_engine.FixedPoseEngine'>
line 9.
line 10.
line 25.
line 27.
line 28.
line 37.
line 38.
line 39.
line 81.
line 82.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 68.
line 69.
line 70.
line 71.
line 72.
line 73.
line 74.
line 40.
line 49.
line 81.
line 82.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
<model name="crate">
  <pose frame="">0 0 0 0 -0 0</pose>
  <static>1</static>
  <allow_auto_disable>0</allow_auto_disable>
  <link name="crate">
    <pose frame="">0 0 0 0 -0 0</pose>
    <collision name="collision">
      <pose frame="">-0 -0 -0.198151 0 -0 0</pose>
      <max_contacts>20</max_contacts>
      <geometry>
        <mesh>
          <uri>file:///data/pcg_gazebo-master/examples/meshes/crate.stl</uri>
          <scale>1 1 1</scale>
        </mesh>
      </geometry>
    </col

Start an instance of Gazebo

In [9]:
try:
    import rospy
    ROS1_AVAILABLE = True
except ImportError:
    ROS1_AVAILABLE = False
    
if not ROS1_AVAILABLE:
    print('No ROS 1 available, source the ROS 1 setup.bash to run Gazebo')
else:
    from pcg_gazebo.generators.creators import create_models_from_config
    from pcg_gazebo.task_manager import Server

    # Start an empty world Gazebo simulation
    server = Server()
    server.create_simulation('default')
    simulation = server.get_simulation('default')
    simulation.create_gazebo_empty_world_task()
    print(simulation.get_task_list())
    print('Is Gazebo running: {}'.format(
        simulation.is_task_running('gazebo')))
    simulation.run_all_tasks()

    # Create a Gazebo proxy
    gazebo_proxy = simulation.get_gazebo_proxy()
    print('ROS configuration:')
    print(gazebo_proxy.ros_config)

    # Pause the simulation
    gazebo_proxy.pause()

2020-09-11 12:06:58,743 | WARNING | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1
['gazebo']
Is Gazebo running: False
2020-09-11 12:06:59,008 | WARNING | __init__ | Error testing Gazebo server, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 12:06:59,013 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 12:06:59,251 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
2020-09-11 12:06:59,484 | WARNING | __init__ | Error testing roscore, message=Command '['rostopic', 'list']' returned non-zero exit status 1
ROS configuration:
ROS_MASTER_URI=http://localhost:19857, GAZEBO_MASTER_URI=http://localhost:28484


Spawn model groups as nested models.

In [10]:
# Spawn models
if ROS1_AVAILABLE:
    for i in range(len(models)):
        print(type(models[i]))
        models[i].spawn(
            robot_namespace='full_crate_{}'.format(i),
            gazebo_proxy=gazebo_proxy, pos=[i * 2 - 2, 0, 0], nested=True)

<class 'pcg_gazebo.simulation.model_group.ModelGroup'>
line 85.
line 86.
line 65.
line 81.
line 82.
line 68.
line 69.
line 70.
line 71.
line 72.
line 73.
line 74.
line 66.
line 67.
line 85.
line 86.
line 65.
line 81.
line 82.
line 68.
line 69.
line 70.
line 71.
line 72.
line 73.
line 74.
line 66.
line 67.
line 85.
line 86.
line 65.
line 81.
line 82.
line 68.
line 69.
line 70.
line 71.
line 72.
line 73.
line 74.
line 66.
line 67.
line 85.
line 86.
line 65.
line 81.
line 82.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 111.
line 68.
line 69.
line 70.
line 71.
line 72.
line 73.
line 74.
line 66.
line 67.
line 85.
line 86.
line 65.
line 81.
line 82.
line 68.
line 69.
line 70.
line 71.
line 72.
line 73.
line 74.
line 66.
line 67.
line 85.
line 86.
line 65.
line 81.
line 82.
line 68.
line 69.
line 70.
line 71.
line 72.
line 73.
line 74.
line 66.
line 67.
line 85.
line 86.
line 65.
line 81.
line 82.
line 68.
line 69.
line 70.
line 71.
line 72.
line 73.
line 74.
l

In [11]:
if ROS1_AVAILABLE:
    simulation.kill_all_tasks()

2020-09-11 12:08:19,592 | WARNING | task | Task <gazebo> - Killing process tree...
2020-09-11 12:08:19,606 | WARNING | task | Sending SIGINT to child process id=4334
2020-09-11 12:08:19,607 | WARNING | task | Child process 4334 still running
2020-09-11 12:08:19,608 | WARNING | task | Sending SIGINT to child process id=4355
2020-09-11 12:08:19,610 | WARNING | task | Child process 4355 still running
2020-09-11 12:08:19,610 | WARNING | task | Sending SIGINT to child process id=4360
2020-09-11 12:08:19,611 | WARNING | task | Child process 4360 still running
2020-09-11 12:08:19,612 | WARNING | task | Sending SIGINT to child process id=4429
2020-09-11 12:08:19,613 | WARNING | task | Child process 4429 still running
2020-09-11 12:08:19,614 | WARNING | task | Sending SIGINT to child process id=4423
2020-09-11 12:08:19,615 | WARNING | task | Child process 4423 still running
2020-09-11 12:08:19,616 | WARNING | task | Sending SIGINT to child process id=4333
2020-09-11 12:08:19,617 | WARNING | tas

A sample of the generated full crates can be seen below

![sim_generated_crates](images/sim_generated_crates.png)